<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/teste_ESTAT%C3%8DSTICO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import wilcoxon
import pandas as pd
import matplotlib.pyplot as plt

# Dados reais dos experimentos (hF scores para cada fold)
data = {
    'GPCR-Pfam': {
        'Sem_Selecao': [57.72, 58.53, 60.42, 58.44, 59.27, 60.41, 59.61, 60.92, 60.29, 58.59],
        'IRH': [58.43, 60.10, 60.64, 58.43, 60.08, 61.28, 59.62, 61.37, 60.30, 60.92],
        'IR': [57.72, 58.74, 60.50, 56.19, 57.78, 58.63, 59.35, 58.43, 59.53, 57.69]
    },
    'GPCR-Prints': {
        'Sem_Selecao': [74.74, 75.44, 76.70, 75.74, 72.30, 75.75, 73.65, 76.48, 79.52, 74.52],
        'IRH': [74.92, 76.09, 77.06, 76.25, 74.61, 76.39, 79.91, 77.32, 80.87, 80.88],
        'IR': [75.16, 77.88, 76.90, 75.50, 75.72, 75.33, 76.16, 76.40, 79.28, 74.69]
    },
    'GPCR-Prosite': {
        'Sem_Selecao': [56.50, 58.81, 57.67, 56.07, 55.23, 57.30, 59.82, 59.68, 57.54, 61.19],
        'IRH': [57.19, 60.25, 57.52, 56.44, 57.99, 58.66, 60.96, 61.72, 59.06, 62.54],
        'IR': [56.50, 59.18, 57.69, 56.11, 55.59, 57.58, 59.88, 59.57, 57.48, 61.25]
    },
    'GPCR-Interpro': {
        'Sem_Selecao': [76.47, 79.94, 76.69, 76.97, 76.05, 76.22, 77.63, 75.45, 77.42, 78.34],
        'IRH': [74.80, 78.82, 74.47, 76.55, 79.15, 78.91, 78.02, 77.91, 80.93, 81.82],
        'IR': [76.47, 79.76, 72.70, 76.86, 75.97, 76.09, 77.76, 74.97, 77.09, 69.30]
    },
    'EC-Pfam': {
        'Sem_Selecao': [90.35, 90.19, 90.19, 91.51, 89.62, 91.00, 89.82, 89.75, 89.78, 89.66],
        'IRH': [92.35, 92.28, 92.22, 93.43, 78.82, 93.45, 92.29, 92.76, 92.14, 91.65],
        'IR': [90.32, 87.73, 90.78, 91.36, 88.03, 90.85, 89.75, 88.90, 89.92, 88.82]
    },
    'EC-Prints': {
        'Sem_Selecao': [92.53, 93.78, 92.87, 92.71, 91.89, 91.27, 91.32, 92.51, 92.16, 92.41],
        'IRH': [94.19, 95.06, 93.12, 94.26, 93.29, 91.10, 93.14, 92.58, 93.35, 94.12],
        'IR': [92.51, 92.60, 92.01, 92.38, 92.81, 91.14, 91.34, 92.32, 93.88, 82.12]
    },
    'EC-Prosite': {
        'Sem_Selecao': [93.70, 92.92, 94.17, 93.53, 94.76, 94.33, 92.52, 93.44, 93.34, 93.77],
        'IRH': [90.97, 96.00, 96.41, 96.08, 96.34, 96.65, 95.84, 95.79, 95.88, 95.21],
        'IR': [92.02, 92.44, 87.71, 91.35, 94.14, 94.08, 92.08, 93.36, 92.07, 93.38]
    },
    'EC-Interpro': {
        'Sem_Selecao': [89.98, 92.43, 91.20, 90.68, 91.44, 91.09, 92.24, 90.91, 92.07, 90.28],
        'IRH': [92.93, 93.68, 93.49, 94.47, 93.10, 95.01, 93.16, 94.78, 93.91, 95.48],
        'IR': [89.82, 93.11, 90.82, 90.57, 91.47, 90.49, 88.78, 91.96, 92.07, 82.99]
    }
}

def wilcoxon_bonferroni(g1, g2, alpha=0.05, n_comparisons=2):
    """
    Teste de Wilcoxon Signed-Rank com correção de Bonferroni

    Parameters:
    -----------
    g1, g2 : arrays pareados
    alpha : nível de significância base (0.05 para 95% confiança)
    n_comparisons : número de comparações (2: Baseline vs IRH, IRH vs IR)

    Returns:
    --------
    statistic, p_value, is_significant
    """
    alpha_corrected = alpha / n_comparisons

    try:
        statistic, p_value = wilcoxon(g1, g2, alternative='two-sided')
        is_significant = p_value < alpha_corrected
        return statistic, p_value, is_significant
    except Exception as e:
        print(f"Erro: {e}")
        return None, None, False

# Construir tabela de resultados
results = []
detailed_analysis = []

for dataset_name, dataset_values in data.items():
    sem = np.array(dataset_values['Sem_Selecao'])
    irh = np.array(dataset_values['IRH'])
    ir = np.array(dataset_values['IR'])

    # Estatísticas descritivas
    sem_mean, sem_std = np.mean(sem), np.std(sem, ddof=1)
    irh_mean, irh_std = np.mean(irh), np.std(irh, ddof=1)
    ir_mean, ir_std = np.mean(ir), np.std(ir, ddof=1)

    # COMPARAÇÃO 1: Sem Seleção vs IRH
    _, p_baseline_irh, sig_baseline_irh = wilcoxon_bonferroni(sem, irh)

    # COMPARAÇÃO 2: IRH vs IR
    _, p_irh_ir, sig_irh_ir = wilcoxon_bonferroni(irh, ir)

    # Adicionar aos resultados
    results.append({
        'Bases de dados': dataset_name,
        'hF (Sem Seleção)': f"{sem_mean:.2f} ({sem_std:.2f})",
        'hF (IRH)': f"{irh_mean:.2f} ({irh_std:.2f})",
        'hF (IR)': f"{ir_mean:.2f} ({ir_std:.2f})",
        'Baseline vs IRH': '★' if sig_baseline_irh else '–',
        'IRH vs IR': '★' if sig_irh_ir else '–'
    })

    # Análise detalhada
    detailed_analysis.append({
        'Base': dataset_name,
        'Baseline vs IRH (p-value)': f"{p_baseline_irh:.6f}" if p_baseline_irh else "N/A",
        'IRH vs IR (p-value)': f"{p_irh_ir:.6f}" if p_irh_ir else "N/A",
        'Ganho IRH vs Baseline (pp)': f"{irh_mean - sem_mean:+.2f}",
        'Ganho IRH vs IR (pp)': f"{irh_mean - ir_mean:+.2f}"
    })

# Criar DataFrame principal
df_results = pd.DataFrame(results)

# Contar "wins" (diferenças significativas)
wins_baseline_irh = df_results['Baseline vs IRH'].value_counts().get('★', 0)
wins_irh_ir = df_results['IRH vs IR'].value_counts().get('★', 0)

# Adicionar linha de wins
wins_row = pd.DataFrame([{
    'Bases de dados': 'Wins (★)',
    'hF (Sem Seleção)': '',
    'hF (IRH)': '',
    'hF (IR)': '',
    'Baseline vs IRH': str(wins_baseline_irh),
    'IRH vs IR': str(wins_irh_ir)
}])

df_final = pd.concat([df_results, wins_row], ignore_index=True)

# ==================== EXIBIÇÃO DOS RESULTADOS ====================

print("\n" + "="*110)
print("Tabela 5.5 – Resultados da avaliação estatística dos cenários de seleção de atributos")
print("="*110)
print("\n📊 Wilcoxon Signed-Rank Test (two-sided) com correção de Bonferroni")
print(f"   Nível de significância: α = 0.05")
print(f"   Número de comparações: 2 (Baseline vs IRH, IRH vs IR)")
print(f"   α corrigido (Bonferroni): 0.05/2 = 0.025")
print("\n")

# Formatar para display
display_cols = {
    'Bases de dados': 'Bases de dados',
    'hF (Sem Seleção)': 'hF (Sem Seleção)\n(desvio padrão)',
    'hF (IRH)': 'hF (IRH)\n(desvio padrão)',
    'hF (IR)': 'hF (IR)\n(desvio padrão)',
    'Baseline vs IRH': 'Resultado do Teste Estatístico (α=0.05)\nBaseline vs IRH',
    'IRH vs IR': 'Resultado do Teste Estatístico (α=0.05)\nIRH vs IR'
}

df_display = df_final.copy()
df_display.columns = display_cols.values()

print(df_display.to_string(index=False))
print("\n" + "="*110)
print("\n📌 Legenda:")
print("   ★ = Diferença estatisticamente significativa (p < 0.025 após correção de Bonferroni)")
print("   – = Diferença não estatisticamente significativa (p ≥ 0.025)")
print("   Os valores entre parênteses representam o desvio padrão")
print("="*110)

# ==================== ANÁLISE DETALHADA ====================

print("\n\n" + "="*100)
print("📈 ANÁLISE DETALHADA DOS P-VALUES E GANHOS")
print("="*100)

df_detailed = pd.DataFrame(detailed_analysis)
print(df_detailed.to_string(index=False))
print(f"\n⚠️  Threshold de significância (α corrigido): 0.025")
print("    pp = pontos percentuais")
print("="*100)

# ==================== RESUMO ESTATÍSTICO ====================

print("\n\n" + "="*100)
print("📊 RESUMO EXECUTIVO")
print("="*100)

print(f"\n✓ Total de bases avaliadas: {len(data)}")
print(f"\n🏆 Comparação 1 (Baseline vs IRH):")
print(f"   - Vitórias do IRH (diferenças significativas): {wins_baseline_irh}/{len(data)}")
print(f"   - Taxa de sucesso: {wins_baseline_irh/len(data)*100:.1f}%")

print(f"\n🏆 Comparação 2 (IRH vs IR):")
print(f"   - Vitórias do IRH (diferenças significativas): {wins_irh_ir}/{len(data)}")
print(f"   - Taxa de sucesso: {wins_irh_ir/len(data)*100:.1f}%")

# Calcular ganhos médios
all_gains_baseline = []
all_gains_ir = []
for ds_name, ds_vals in data.items():
    sem = np.mean(ds_vals['Sem_Selecao'])
    irh = np.mean(ds_vals['IRH'])
    ir = np.mean(ds_vals['IR'])
    all_gains_baseline.append(irh - sem)
    all_gains_ir.append(irh - ir)

print(f"\n📈 Ganhos médios do IRH:")
print(f"   - vs Baseline: {np.mean(all_gains_baseline):+.2f} pp (±{np.std(all_gains_baseline):.2f})")
print(f"   - vs IR Plana: {np.mean(all_gains_ir):+.2f} pp (±{np.std(all_gains_ir):.2f})")

print("="*100)






Tabela 5.5 – Resultados da avaliação estatística dos cenários de seleção de atributos

📊 Wilcoxon Signed-Rank Test (two-sided) com correção de Bonferroni
   Nível de significância: α = 0.05
   Número de comparações: 2 (Baseline vs IRH, IRH vs IR)
   α corrigido (Bonferroni): 0.05/2 = 0.025


Bases de dados hF (Sem Seleção)\n(desvio padrão) hF (IRH)\n(desvio padrão) hF (IR)\n(desvio padrão) Resultado do Teste Estatístico (α=0.05)\nBaseline vs IRH Resultado do Teste Estatístico (α=0.05)\nIRH vs IR
     GPCR-Pfam                      59.42 (1.07)              60.12 (1.04)             58.46 (1.20)                                                        ★                                                  ★
   GPCR-Prints                      75.48 (1.94)              77.43 (2.32)             76.30 (1.40)                                                        ★                                                  –
  GPCR-Prosite                      57.98 (1.87)              59.23 (2.05)        